# EDS Block Experimentation - Context Aware

This notebook works in both JSLab (Node.js) and browser environments.

## Environment Detection

- **JSLab/Node.js**: Sets up jsdom virtual DOM and helper functions
- **Browser**: Uses native browser APIs directly

## Important Notes

1. **In JSLab**: Run Cell 1 first to initialize the environment
2. **In Browser (ipynb-viewer)**: Skip Cell 1, it's not needed! Start from Cell 2.
3. **Pure EDS blocks only** - Works best with vanilla JavaScript blocks
4. **Context-aware execution** - Automatically detects Node.js vs browser
5. **Visual output** - In JSLab, saves HTML files to `ipynb-tests/` folder

In [ ]:
// ============================================================================
// SETUP: Context-aware initialization
// BROWSER USERS: Skip this cell! It's only for JSLab/Node.js
// ============================================================================

// Detect execution environment
const isNode = typeof process !== 'undefined' && process.versions && process.versions.node;
const isBrowser = typeof window !== 'undefined' && typeof document !== 'undefined';

console.log('Environment:', isNode ? 'Node.js (JSLab)' : 'Browser');

// ============================================================================
// NODE.JS SETUP (JSLab/Jupyter)
// ============================================================================
if (isNode) {
  // Load jsdom and set up virtual DOM
  const { JSDOM } = require('jsdom');
  
  // Create virtual DOM
  const dom = new JSDOM('<!DOCTYPE html><html><head></head><body></body></html>', {
    url: 'http://localhost',
    pretendToBeVisual: true
  });
  
  // Make DOM globals available
  global.document = dom.window.document;
  global.window = dom.window;
  global.HTMLElement = dom.window.HTMLElement;
  global.Element = dom.window.Element;
  global.Node = dom.window.Node;
  global.customElements = dom.window.customElements;
  global.CustomEvent = dom.window.CustomEvent;
  global.Event = dom.window.Event;
  
  console.log('✓ Virtual DOM environment initialized');
  
  // Ensure output directory exists
  const fs = require('fs');
  const outputDir = './ipynb-tests';
  if (!fs.existsSync(outputDir)) {
    fs.mkdirSync(outputDir, { recursive: true });
    console.log(`✓ Created output directory: ${outputDir}`);
  }
  
  // Load helper functions from external module
  const path = require('path');
  const helpersPath = path.resolve('./scripts/ipynb-helpers.js');
  
  // Dynamic import of ES modules
  import(helpersPath).then(helpers => {
    global.loadBlockStyles = helpers.loadBlockStyles;
    global.testBlock = helpers.testBlock;
    global.saveBlockHTML = helpers.saveBlockHTML;
    
    console.log('✓ Loaded helper functions from scripts/ipynb-helpers.js');
    console.log('✓ Available functions:');
    console.log('  - global.loadBlockStyles(blockName)');
    console.log('  - global.testBlock(blockName, innerHTML)');
    console.log('  - global.saveBlockHTML(blockName, innerHTML, filename, options)');
  }).catch(error => {
    console.error('✗ Failed to load helpers:', error.message);
    console.log('ℹ Falling back to inline helpers...');
    
    // Fallback: Define minimal inline helpers if external file fails
    global.testBlock = async function(blockName, innerHTML = '') {
      console.error('Helper functions not loaded. Please check scripts/ipynb-helpers.js exists.');
      return null;
    };
  });
  
  console.log('✓ Node.js environment ready');
  console.log('✓ Helper functions loading...');
}

// ============================================================================
// BROWSER SETUP
// ============================================================================
else if (isBrowser) {
  console.log('✓ Browser environment detected');
  console.log('✓ Using native browser APIs');
  console.log('ℹ️  Skip Cell 1 in browser - not needed!');
  console.log('ℹ️  Start running cells from Cell 2 onwards');
  
  // Browser helpers use native APIs
  window.testBlock = async function(blockName, innerHTML = '') {
    console.log(`Testing: ${blockName}`);
    
    const block = document.createElement('div');
    block.className = blockName;
    
    if (innerHTML) {
      block.innerHTML = innerHTML;
    }
    
    console.log('Block created:', block);
    return block;
  };
  
  console.log('✓ Browser helpers ready');
}

console.log('\n========================================');
console.log('Setup complete!');
console.log('========================================\n');

'Setup complete!'

## Part 1: Simple Tests

Test basic DOM functionality that works in both environments.

In [ ]:
// Context-aware DOM test
const isNode = typeof process !== 'undefined' && process.versions?.node;
const doc = isNode ? global.document : document;

const testDiv = doc.createElement('div');
testDiv.textContent = 'Hello from ' + (isNode ? 'Node.js' : 'Browser') + '!';
testDiv.className = 'test';

console.log('✓ Created element:', testDiv.outerHTML);
testDiv.outerHTML

## Part 2: Simple Calculations

Pure JavaScript that works everywhere.

In [ ]:
// Simple calculation
const a = 10;
const b = 20;
console.log('Sum:', a + b);
return a + b;

In [ ]:
// Array operations
const numbers = [1, 2, 3, 4, 5];
const doubled = numbers.map(n => n * 2);
console.log('Original:', numbers);
console.log('Doubled:', doubled);
return doubled;

## Part 3: String Manipulation

In [ ]:
// Text transformations
const text = 'hello world from javascript';

console.log('Original:', text);
console.log('Uppercase:', text.toUpperCase());
console.log('Title Case:', text.replace(/\b\w/g, c => c.toUpperCase()));

return text.toUpperCase();

## Part 4: Mathematical Operations

In [ ]:
// Calculate factorial
function factorial(n) {
  if (n <= 1) return 1;
  return n * factorial(n - 1);
}

const result = factorial(5);
console.log('5! =', result);
return result;

In [ ]:
// Fibonacci sequence
function fibonacci(n) {
  const seq = [0, 1];
  for (let i = 2; i < n; i++) {
    seq[i] = seq[i - 1] + seq[i - 2];
  }
  return seq;
}

const fib = fibonacci(10);
console.log('First 10 Fibonacci numbers:', fib);
return fib;

## Part 5: DOM Interaction (Context-Aware)

In [ ]:
// Context-aware DOM query
const isNode = typeof process !== 'undefined' && process.versions?.node;
const doc = isNode ? global.document : document;

// Count blocks on the page
const blockCount = doc.querySelectorAll('.block').length;
console.log('Number of blocks on page:', blockCount);
return `Found ${blockCount} blocks`;

## Part 6: Error Handling

In [ ]:
// Error handling example
try {
  const result = 100 / 5;
  console.log('Result:', result);
  return result;
} catch (error) {
  console.error('Error occurred:', error.message);
  return null;
}

## Part 7: Object Creation

In [ ]:
// Create and manipulate objects
const person = {
  name: 'John Doe',
  age: 30,
  city: 'San Francisco',
  greet() {
    return `Hello, I'm ${this.name} from ${this.city}`;
  }
};

console.log('Person:', person);
console.log('Greeting:', person.greet());

return person;

## Part 8: Testing EDS Blocks (Node.js Only)

These cells only work in JSLab/Node.js environment.

In [ ]:
// Only run in Node.js
const isNode = typeof process !== 'undefined' && process.versions?.node;

if (isNode) {
  // Test HelloWorld block
  const helloBlock = await global.testBlock('helloworld');
  console.log('Block HTML:', helloBlock.outerHTML);
  helloBlock.outerHTML;
} else {
  console.log('⚠ This cell requires Node.js environment');
  'Skipped in browser';
}

In [ ]:
// Only run in Node.js - List available blocks
const isNode = typeof process !== 'undefined' && process.versions?.node;

if (isNode) {
  const fs = require('fs').promises;
  const blocksDir = './blocks';
  
  const entries = await fs.readdir(blocksDir, { withFileTypes: true });
  const blocks = entries
    .filter(entry => entry.isDirectory())
    .map(entry => entry.name)
    .sort();
  
  console.log(`Found ${blocks.length} blocks:\n`);
  console.log(blocks.join('\n'));
  
  blocks;
} else {
  console.log('⚠ This cell requires Node.js environment');
  'Skipped in browser';
}

## Quick Reference

### Environment Detection
```javascript
const isNode = typeof process !== 'undefined' && process.versions?.node;
const isBrowser = typeof window !== 'undefined';
```

### Context-Aware DOM Access
```javascript
const doc = isNode ? global.document : document;
```

### Node.js Helpers (JSLab only)

**Test blocks (returns DOM element):**
- `global.testBlock(blockName, innerHTML)` - Test a block and return DOM

**Save with Live Preview (NEW!):**
- `global.saveBlockHTML(blockName, innerHTML)` - Creates TWO files:
  - `blockname-preview.html` - The actual styled block
  - `blockname-live-preview.html` - **Opens in browser with iframe controls!**

**Live Preview Features:**
- 🔴 Dark themed wrapper with controls
- ↻ Refresh button to reload preview
- ✕ Close button (or press ESC key)
- Status bar showing file location
- Fullscreen iframe display

**Disable live preview:**
```javascript
await global.saveBlockHTML('blockname', '<div>content</div>', null, { livePreview: false });
```

**Load styles manually:**
- `global.loadBlockStyles(blockName)` - Load CSS for a block

### Browser Helpers
- `window.testBlock(blockName, innerHTML)` - Create block element

### Example Usage (Node.js)

```javascript
// Create an accordion with live preview
const accordionContent = `
  <div>
    <div>Question 1</div>
    <div>Answer 1</div>
  </div>
`;

// This creates BOTH files automatically:
// - accordion-preview.html (actual block)
// - accordion-live-preview.html (iframe wrapper with controls)
await global.saveBlockHTML('accordion', accordionContent);

// Then open: ipynb-tests/accordion-live-preview.html in your browser!
```

### Tips
1. Always run **Cell 1** first to initialize the environment
2. Pure JavaScript works in both environments
3. DOM operations need context-aware code
4. Block testing only works in Node.js environment
5. Browser execution is for simple calculations and demos
6. **NEW**: Live preview creates an interactive iframe wrapper automatically!
7. Open the `-live-preview.html` file for best visual testing experience